# Python Native __slots__ Attribute Exploration
> Diving into Python to see what __slots__ does and how it can be used

- toc: true
- branch: master
- badges: true
- comments: true
- author: Kevin Bird
- categories: [python, technical, exploration]

In [1]:
#hide
import sys

## Introduction

While reading the [python data model](https://docs.python.org/3.8/reference/datamodel.html#slots) documentation, I came across something I hadn't seen before. `__slots__` is an optional argument that allows users to "explicitly declare data members".  It is an interesting concept that I haven't seen utilized, but perhaps the reason is that not many people are aware it exists.  I am going to explore this attribute that is available to see if it might provide value for my future projects.  According to [this blog post](https://book.pythontips.com/en/latest/__slots__magic.html), \_\_slots\_\_ can significantly reduce the amount of ram required to create objects (40-50%!).  Now let's dive in and figure out how it's used!

In [2]:
#hide_input
print("Python Version Check:\n" + sys.version)

Python Version Check:
3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


## Example #1: Typical Case

The first example we look at is the working example.  we will have a class `A1` with __slots__ set to accept one var named `var1`.

In [3]:
class A1:
    __slots__ = ['var1']
    def __init__(self, value_passed_through_here):
        self.var1 = value_passed_through_here

In [5]:
a1 = A1(1); a1.var1

1

`a` has been created and everything is going well.  Let's try adding another attribute. 

In [6]:
a1.var2 = "but can I set another var?"

AttributeError: 'A1' object has no attribute 'var2'

`a.var2` fails as expected because it isn't in the `__slots__` list and `__slots__` is read-only so it cannot be updated. 

In [7]:
a1.__slots__ = ['var2']

AttributeError: 'A1' object attribute '__slots__' is read-only

In [8]:
a1.__slots__ = ['var1', 'var2']

AttributeError: 'A1' object attribute '__slots__' is read-only

When \_\_slots\_\_ is used, the \_\_dict\_\_ value is not set.  Let's explore that a little further though.  

In [9]:
a1.__dict__

AttributeError: 'A1' object has no attribute '__dict__'

## Example #2a: Exploring \_\_dict\_\_ Without Using \_\_slots\_\_

In [10]:
class A2A:
    def __init__(self, value_passed_through_here):
        self.var1 = value_passed_through_here

In [11]:
a2a = A2A(1)

var1 shows up as expected when creating an object

In [12]:
a2a.__dict__

{'var1': 1}

In [13]:
a2a.var2 = 'adding a second thing'

Adding a second variable adds it to the \_\_dict\_\_ as expected

In [14]:
a2a.__dict__

{'var1': 1, 'var2': 'adding a second thing'}

## Example #2b: Exploring \_\_dict\_\_ When Using \_\_slots\_\_

In [15]:
class A2B:
    __slots__ = ['var1', '__dict__']
    def __init__(self, value_passed_through_here):
        self.var1 = value_passed_through_here

In [16]:
a2b = A2B(1)

In [17]:
a2b.__dict__

{}

\_\_dict\_\_ exists now since we added it to \_\_slots\_\_, **but it isn't populating the \_\_dict\_\_ like normal**.  We are still able to call the attribute `var1` though.

In [18]:
a2b.var1

1

In [19]:
a2b.var2 = "test if we can add new variables now"

Surprisingly, once we add \_\_dict\_\_ to the \_\_slots\_\_ list, adding a new var works.

In [20]:
a2b.__dict__

{'var2': 'test if we can add new variables now'}

When we look at \_\_dict\_\_ after adding var2, there is an entry in \_\_dict\_\_ as well.  

In [21]:
a2b.var2

'test if we can add new variables now'

So if we enable \_\_dict\_\_ we are able to add new items to the \_\_dict\_\_, but \_\_dict\_\_ has to be explicitly defined to work.  

## Example 3: Inheritance

Now that we've explored \_\_slots\_\_, let's see how it behaves when one class is inherited from another.  

In [22]:
class A3:
    __slots__ = ['a']
    def __init__(self, a):
        self.a = a

In [23]:
a3 = A3(1); a3.a

1

In [24]:
class B3A(A3):
    def __init__(self):
        self.a = 1
        self.b = 2

In [25]:
b3a = B3A()

In [26]:
b3a.__slots__

['a']

In [27]:
b3a.__dict__

{'b': 2}

In [28]:
b3a.a

1

In [29]:
b3a.c = "can I set c?"

In [30]:
b3a.__dict__

{'b': 2, 'c': 'can I set c?'}

So when `B3A` is inherited from `A3`, it uses the __slots__ class, but it also reverts back in a lot of ways to a normal, non-slots, class again.  The last thing I'm going to try is actually setting a `__slots__` in `B3B` just to see what happens

In [31]:
class B3B(A3):
    __slots__ = ['a','b']
    def __init__(self):
        self.a = 1
        self.b = 2

In [32]:
b3b = B3B()

In [33]:
b3b.c = "can I set this?"

AttributeError: 'B3B' object has no attribute 'c'

So now that we have given `B3B` a `__slots__` it is no longer able to behave the same way that `B3A` is.  

## Conclusion

\_\_slots\_\_ is an interesting concept that is built into Python that I hadn't heard of and wanted to explore.  Hopefully this notebook is informative to other Python users as well.  Things didn't always behave as I would have expected and that's part of the fun of actually testing out the code to see how things work in practice.  